# 규제 선형 모델

- 다항식이 복잡해지면서 회귀 계수값이 크게 설정되고 과대 적합을 유발함
    -> 훈련 데이터에서는 높은 성능을 보이지만, 테스트 데이터에서는 낮은 성능을 보임

- 선형 모델의 목적은 비용 함수를 최소화하는 것
- 이때, 규제 선형 모델은 비용 함수를 최소화하는 데에 패널티(alpha)를 적용해 회귀 계수의 크기를 제한함


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Ridge 회귀
- L2 규제를 적용한 회귀 모델
- 회귀 계수들의 제곱합에 alpha 패널티를 곱해 비용 산정에 반영
- alpha 값이 커지면 회귀 계수가 작아짐 -> 과대적합 방지